In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [33]:
import pandas as pd
import numpy as np

def load_and_process_data(data1, data2, data3, data4, common_key, target_column):
    """Charge, fusionne et nettoie les fichiers CSV."""
    caract = pd.read_csv(data1, sep=";", low_memory=False)
    lieux = pd.read_csv(data2, sep=";", low_memory=False)
    usagers = pd.read_csv(data3, sep=";", low_memory=False)
    vehicules = pd.read_csv(data4, sep=";", low_memory=False)
    
    merged_df = caract.merge(lieux, on=common_key, how="inner") \
                      .merge(usagers, on=common_key, how="inner") \
                      .merge(vehicules, on=common_key, how="inner")
    
    print(f"Colonnes disponibles après fusion : {merged_df.columns.tolist()}")

    merged_df = merged_df.dropna(axis=1, how='all')
    merged_df = merged_df.loc[:, merged_df.nunique() > 1]
    
    y = merged_df[target_column]
    X = merged_df.drop(columns=[target_column, common_key])
    return X, y

def correlation_matrix(X, y):
    """Calcule la corrélation entre chaque colonne numérique et la cible."""
    correlations = {}
    for col in X.columns:
        if X[col].dtype != 'object':
            corr = np.corrcoef(X[col], y)[0, 1]  # Corrélation Pearson
            correlations[col] = corr
    sorted_corr = sorted(correlations.items(), key=lambda x: abs(x[1]), reverse=True)
    return sorted_corr

def categorical_analysis(X, y):
    """Analyse l'impact des variables catégorielles sur la cible."""
    X_with_target = X.copy()
    X_with_target['target'] = y

    category_means = {}
    for col in X.columns:
        if X[col].dtype == 'object':
            category_means[col] = X_with_target.groupby(col)['target'].mean()
    return category_means

def low_variance_filter(X, threshold=0.01):
    """Supprime les colonnes numériques avec une variance inférieure à un seuil."""
    X_numeric = X.select_dtypes(include=[np.number])
    variances = X_numeric.var()
    important_columns = variances[variances > threshold].index    
    return X[important_columns]

def auto_handle_nan(df, nan_threshold_delete=0.5, nan_threshold_impute=0.1):
    """
    Traite automatiquement les valeurs NaN dans un dataset.
    - Supprime les colonnes avec trop de NaN.
    - Impute (remplace) les NaN avec des stratégies adaptées :
      - Moyenne pour colonnes numériques.
      - Mode ou "Inconnu" pour colonnes catégorielles.
    """
    print("Analyse des NaN dans le dataset...\n")
    
    nan_percent = df.isnull().mean()
    print("Pourcentage de valeurs manquantes par colonne :")
    print(nan_percent)
    
    cols_to_delete = nan_percent[nan_percent > nan_threshold_delete].index
    print(f"\nColonnes supprimées (trop de NaN > {nan_threshold_delete*100}%): {list(cols_to_delete)}")
    df = df.drop(columns=cols_to_delete)
    
    for col in df.columns:
        missing = df[col].isnull().sum()
        if missing > 0:
            if df[col].dtype == 'object':
                if nan_percent[col] > nan_threshold_impute:
                    print(f"Colonne '{col}' : Imputation avec 'Manquant' (catégorielle)")
                    df[col] = df[col].fillna("Manquant")
                else:
                    print(f"Colonne '{col}' : Imputation avec la valeur la plus fréquente (mode)")
                    df[col] = df[col].fillna(df[col].mode()[0])
            else:
                if nan_percent[col] > nan_threshold_impute:
                    print(f"Colonne '{col}' : Imputation avec la médiane (numérique)")
                    df[col] = df[col].fillna(df[col].median())
                else:
                    print(f"Colonne '{col}' : Imputation avec la moyenne (numérique)")
                    df[col] = df[col].fillna(df[col].mean())
    
    print("\nTraitement des NaN terminé.")
    return df


In [57]:
def preprocess_data(data1, data2, data3, data4, common_key, target_column):
    print("Chargement et fusion des données...")
    X, y = load_and_process_data(data1, data2, data3, data4, common_key, target_column)

    print("\nTraitement des valeurs NaN...")
    X = auto_handle_nan(X)

    print("\nAnalyse des corrélations avec la cible :")
    sorted_corr = correlation_matrix(X, y)
    for col, corr in sorted_corr:
        print(f"{col}: {corr:.2f}")

    print("\nAnalyse des variables catégorielles :")
    cat_analysis = categorical_analysis(X, y)
    for col, means in cat_analysis.items():
        print(f"\nColonne : {col}")
        print(means)
    
    print("\nFiltrage des variables à faible variance...")
    X_filtered = low_variance_filter(X)
    print(f"Colonnes restantes après filtrage : {X_filtered.columns.tolist()}")

    print("\nRésumé des colonnes importantes pour la prédiction :")
    print(f"Nombre de colonnes finales : {len(X_filtered.columns)}")
    print(X_filtered.head())
    data = X_filtered
    
    print("Nombre de doublons avant suppression :", data.duplicated().sum())
    print("Lignes dupliquées :")
    print(data[data.duplicated()])

    data = data.drop_duplicates()
    print("Nombre de doublons après suppression :", data.duplicated().sum())
    return(data)

data = preprocess_data("data/caract-2023.csv", "data/lieux-2023.csv", "data/usagers-2023.csv", "data/vehicules-2023.csv", "Num_Acc", "agg")
data

Chargement et fusion des données...
Colonnes disponibles après fusion : ['Num_Acc', 'jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'com', 'agg', 'int', 'atm', 'col', 'adr', 'lat', 'long', 'catr', 'voie', 'v1', 'v2', 'circ', 'nbv', 'vosp', 'prof', 'pr', 'pr1', 'plan', 'lartpc', 'larrout', 'surf', 'infra', 'situ', 'vma', 'id_usager', 'id_vehicule_x', 'num_veh_x', 'place', 'catu', 'grav', 'sexe', 'an_nais', 'trajet', 'secu1', 'secu2', 'secu3', 'locp', 'actp', 'etatp', 'id_vehicule_y', 'num_veh_y', 'senc', 'catv', 'obs', 'obsm', 'choc', 'manv', 'motor', 'occutc']

Traitement des valeurs NaN...
Analyse des NaN dans le dataset...

Pourcentage de valeurs manquantes par colonne :
jour             0.000000
mois             0.000000
hrmn             0.000000
lum              0.000000
dep              0.000000
com              0.000000
int              0.000000
atm              0.000000
col              0.000000
adr              0.025318
lat              0.000000
long             0.000000
catr      

,jour,mois,lum,int,atm,col,catr,v1,circ,vosp,...,secu3,locp,etatp,senc,catv,obs,obsm,choc,manv,motor
0,7,5,1,4,2,7,4,0,1,0,...,-1,-1,-1,1,30,0,0,5,1,1
2,7,5,5,1,3,6,3,0,2,2,...,-1,-1,-1,2,7,0,1,1,1,1
3,7,5,5,1,3,6,3,0,2,2,...,-1,2,1,2,7,0,1,1,1,1
4,7,5,1,3,2,1,3,0,2,0,...,0,0,-1,1,2,0,2,1,16,1
5,7,5,1,3,2,1,3,0,2,0,...,0,0,-1,2,7,0,2,2,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309336,20,10,1,1,6,3,4,0,2,2,...,-1,-1,-1,2,50,14,2,1,1,3
309337,20,10,1,1,6,3,4,0,2,2,...,-1,-1,-1,2,7,0,2,7,22,1
309338,20,10,1,1,6,3,4,0,2,2,...,-1,-1,-1,2,50,14,2,1,1,3
309339,20,10,1,1,6,3,4,0,2,2,...,-1,-1,-1,2,7,0,2,7,22,1
